# Decision trees

In [3]:
!pip install statsmodels==0.13.5

import numpy as np                     # Llibreria matematica
import matplotlib.pyplot as plt        # Per mostrar plots
import sklearn                         # Llibreia de DM
import sklearn.datasets as ds          # Per carregar més facilment el dataset digits
import sklearn.model_selection as cv   # Pel Cross-validation
import sklearn.neighbors as nb         # Per fer servir el knn
import pandas as pd
from sklearn.naive_bayes import GaussianNB       ### Because continuous data

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB  # For binari features (f.i. word appears or not in document)
from statsmodels.stats.proportion import proportion_confint
from sklearn.utils import shuffle
from sklearn.metrics import classification_report
from sklearn import tree

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
data = pd.read_csv('fake_job_preprocessed.csv')
data

,title,location,department,salary_range,company_profile,description,requirements,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,6.333616e-09,0.041026,0.000000e+00,2.222222e-02,0.000051,0.056281,0.056281,0,1,0,0.166667,0.068627,0.102041,4.431315e-03,0.017964,0
1,3.003079e-01,0.380666,6.413994e-02,5.628121e-02,0.144389,0.056281,0.015136,0,0,0,0.041833,0.039746,0.012579,3.703704e-02,0.032536,0
2,5.628121e-02,0.000000,5.628121e-02,1.513635e-02,0.015136,0.056281,0.056281,0,1,1,0.050260,0.030513,0.027523,0.000000e+00,0.019231,0
3,1.513635e-02,0.002669,6.413994e-02,1.160042e-10,0.144389,0.056281,0.056281,0,0,0,0.050260,0.030513,0.017430,1.732102e-02,0.015110,0
4,5.628121e-02,0.000003,3.430079e-02,1.391622e-04,0.015136,0.056281,0.056281,0,1,1,0.050260,0.145455,0.017430,7.816336e-13,0.017516,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5201,5.628121e-02,0.003155,3.430079e-02,3.153322e-10,0.000019,0.056281,0.056281,0,1,1,0.050260,0.132733,0.017430,1.911765e-01,0.017516,0
5202,3.806656e-01,0.056281,3.430079e-02,3.003079e-01,0.144389,0.056281,0.380666,0,1,0,0.050260,0.132733,0.065969,1.276596e-01,0.017516,0
5203,0.000000e+00,0.021277,2.272727e-02,0.000000e+00,0.000377,0.056281,0.000377,0,1,0,0.050260,0.039746,0.017430,1.879699e-03,0.000000,0
5204,6.944797e-06,0.057692,1.391622e-04,3.749082e-17,0.144389,0.002669,0.002669,0,0,0,0.050260,0.030513,0.017430,4.431315e-03,0.015110,0


In [3]:
X = data.values[:, 0:data.shape[1]-1]
y = data.values[:, data.shape[1]-1]

X, y = shuffle(X, y)

# Let's do a simple cross-validation: split data into training and test sets (test 30% of data)
(X_train, X_test,  y_train, y_test) = cv.train_test_split(X, y, test_size=.33, random_state=1)

### Mejores parámetros

In [4]:
from sklearn.model_selection import RandomizedSearchCV

In [5]:
# Define the decision tree classifier
dt = tree.DecisionTreeClassifier()

# Define the parameters that you want to optimize
parameters = {'criterion': ['gini', 'entropy', 'log_loss'],
              'splitter': ['best', 'random'],
              'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
              'min_samples_split': [1,2,3,4,5],
              'max_features': [4,5,6,7]}

# Create a randomized search object using the decision tree classifier and the parameters dictionary
rs = RandomizedSearchCV(dt, parameters, cv=10, scoring='f1')

In [6]:
rs.fit(X_train, y_train)

/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
30 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/tree/_classes.py", line 969, in fit
    super().fit(
  File "/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/tree/_classes.py", line 265, in fit
    check_scalar(
  File "/shared-libs/python3.9/py/lib/py

RandomizedSearchCV(cv=10, estimator=DecisionTreeClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy',
                                                      'log_loss'],
                                        'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                      10],
                                        'max_features': [4, 5, 6, 7],
                                        'min_samples_split': [1, 2, 3, 4, 5],
                                        'splitter': ['best', 'random']},
                   scoring='f1')

In [7]:
print(rs.best_score_)
print(rs.best_params_)
BestPar = rs.best_params_

0.7588694844920776
{'splitter': 'best', 'min_samples_split': 5, 'max_features': 7, 'max_depth': 5, 'criterion': 'entropy'}


In [8]:
clf = tree.DecisionTreeClassifier(criterion=BestPar['criterion'], max_depth=BestPar['max_depth'],
                                max_features=BestPar['max_features'],
                                min_samples_split=BestPar['min_samples_split'], splitter=BestPar['splitter'])
pred = clf.fit(X_train, y_train).predict(X_test)
print('Confusion Matrix:')
print(sklearn.metrics.confusion_matrix(y_test, pred))
print()
print(sklearn.metrics.classification_report(y_test, pred))

## Print tree
fig = plt.gcf()
fig.set_size_inches(200, 100)

col = data.columns
col = col[0:15]

tree.plot_tree(clf, filled=True,rounded=True,feature_names=list(col))
fig.savefig('tree.png', bbox_inches='tight')

plt.axis([0, 0.5, 0, 0.5])
plt.show()

Confusion Matrix:
[[1602   13]
 [  35   68]]

              precision    recall  f1-score   support

         0.0       0.98      0.99      0.99      1615
         1.0       0.84      0.66      0.74       103

    accuracy                           0.97      1718
   macro avg       0.91      0.83      0.86      1718
weighted avg       0.97      0.97      0.97      1718



In [9]:
for colName in data:
    print(colName, data.columns.get_loc(colName))

title 0
location 1
department 2
salary_range 3
company_profile 4
description 5
requirements 6
telecommuting 7
has_company_logo 8
has_questions 9
employment_type 10
required_experience 11
required_education 12
industry 13
function 14
fraudulent 15


In [10]:
cont = 0
for element in y_test:
    if element == 1:
        break
    cont += 1
cont

3

In [34]:
print(X_test[3][6])
print(y_test[3])

0.0562812139838647
1.0


### Fiabilidad del arbol con las hojas

In [40]:
count = 0
for i in range(0, len(X_test)):
    if(X_test[i][4] > 0.1 and X_test[i][3] > 0.058 and X_test[i][13] > 0.067 and X_test[i][1] > 0.029 and 
       X_test[i][6] > 0.052):
        count += 1
print(count)

62


In [18]:
count2 = 0
for i in range(0, len(X_test)):
    if(X_test[i][4] <= 0.1 and X_test[i][4] <= 0.036):
        count2 += 1
print(count2)

1109


In [37]:
count3 = 0
for i in range(0, len(X_test)):
    if(X_test[i][4] <= 0.1 and X_test[i][4] > 0.036 and X_test[i][0] > 0.087
        and X_test[i][8] > 0.5 and X_test[i][3] > 0.135):
        count3 += 1
print(count3)

5


### Con cross-validation

In [3]:
X = data.values[:, 0:data.shape[1]-1]
y = data.values[:, data.shape[1]-1]

In [4]:
from sklearn import metrics
(X_train, X_test,  y_train, y_test) = cv.train_test_split(X, y, test_size=.3, random_state=1)

BestPar = {'splitter': 'best', 'min_samples_split': 5, 'max_features': 7, 'max_depth': 5, 'criterion': 'entropy'}
# Create a DT classifier object
knc = tree.DecisionTreeClassifier(criterion=BestPar['criterion'], max_depth=BestPar['max_depth'],
                                max_features=BestPar['max_features'],
                                min_samples_split=BestPar['min_samples_split'], splitter=BestPar['splitter'])

# Train the classifier
knc.fit(X_train, y_train)
y_pred = knc.predict(X_test)
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.98      0.99      0.99      1467
         1.0       0.89      0.72      0.80        95

    accuracy                           0.98      1562
   macro avg       0.94      0.86      0.89      1562
weighted avg       0.98      0.98      0.98      1562



<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=91572d65-ca52-49ec-8072-8fb246390be4' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>